# Importing and Using SOFA Plugins
This tutorial explains how to extend SOFA's functionality by loading plugins. You will learn the difference between the various loading methods and how to ensure your components are correctly registered.

### Learning Objectives
- Understand the role of **Plugins** and the **Component Factory** in SOFA.
- Use `SofaRuntime` to load plugins programmatically.
- Use the `RequiredPlugin` component for scene-graph-embedded dependencies.
- Understand the timing requirements for plugin loading.

---

## 1. What are SOFA Plugins?
In SOFA, a **plugin** is a dynamic library (module) that contains reusable components such as physics solvers, force fields, visualization engines, or constraints. 

SOFA uses a **Component Factory** (a registry). When a plugin is loaded, it registers its classes into this factory. Once registered, these components become available to be added to your Scene Graph via `addObject()`.

**Critical Rule**: A plugin must be loaded **before** you attempt to create any component that belongs to it. If you try to add a component from an unloaded plugin, SOFA will return an error stating that the component type is unknown.

---

## 2. Environment Setup
As always, we start by importing the core modules.

In [ ]:
import Sofa
import SofaRuntime

# Initialize the SOFA runtime
SofaRuntime.init()

## 3. Loading Plugins via `SofaRuntime`
The `SofaRuntime` module provides direct Python functions to manage plugins. This method is often used for global configuration or when running simulations from a custom Python script.

### `importPlugin`
This is the most common way to load a specific plugin by its name.

In [ ]:
# Load the visualization plugin
SofaRuntime.importPlugin("Sofa.Component.Visual")
print("Plugin 'Sofa.Component.Visual' loaded.")

### `auto_load_plugins`
SOFA can also attempt to automatically load all plugins found in its plugin directories. While convenient, it may increase initialization time.

In [ ]:
# SofaRuntime.auto_load_plugins()
# print("All known plugins attempted to load.")

## 4. The `RequiredPlugin` Component
An alternative (and often preferred) method is to add a `RequiredPlugin` component directly into the Scene Graph.

### Why use `RequiredPlugin`?
- **Portability**: The scene file itself declares its dependencies. If you share your script, the next user (or `runSofa`) will know exactly which plugins are needed.
- **Compatibility**: It works seamlessly with `runSofa` and `.scn` files.

### Usage
You typically add `RequiredPlugin` at the very top of your `root` node.

In [ ]:
root = Sofa.Core.Node("root")

# Declare a dependency on the Backward ODE Solver plugin
root.addObject("RequiredPlugin", pluginName="Sofa.Component.ODESolver.Backward")

## 5. Full Educational Example
This example demonstrates a complete simulation setup where multiple plugins are required. We will use the **Hybrid Script** pattern introduced in the previous tutorial.

In [ ]:
def createScene(rootNode):
    """
    Creates a scene that requires multiple plugins for implicit integration.
    """
    # 1. Declare all required plugins first
    rootNode.addObject("RequiredPlugin", pluginName=[
        "Sofa.Component.ODESolver.Backward", # For EulerImplicitSolver
        "Sofa.Component.LinearSolver.Direct",   # For EigenSimplicialLDLT
        "Sofa.Component.StateContainer",      # For MechanicalObject
        "Sofa.Component.Mass"                 # For UniformMass
    ])

    # 2. Add the Animation Loop
    rootNode.addObject("DefaultAnimationLoop")

    # 3. Add Physics Components (from the loaded plugins)
    rootNode.addObject("EulerImplicitSolver")
    rootNode.addObject("EigenSimplicialLDLT", template='CompressedRowSparseMatrixMat3x3')
    
    # 4. Add the Particle
    rootNode.addObject("MechanicalObject", name="particle")
    rootNode.addObject("UniformMass", vertexMass=1.0)

    return rootNode

def main():
    # Standard standalone execution logic
    import SofaRuntime
    SofaRuntime.init()
    
    root = Sofa.Core.Node("root")
    createScene(root)
    
    Sofa.Simulation.initRoot(root)
    
    print(f"Simulation initialized with {len(root.children)} nodes.")
    
    # Step through the simulation
    for iteration in range(5):
        Sofa.Simulation.animate(root, root.dt.value)
        print(f"Step {iteration+1}: Time = {root.time.value:.3f}")

if __name__ == '__main__':
    main()

### Summary of Best Practices
1. **Be Specific**: Only load the plugins you actually need to keep the simulation lightweight.
2. **Order Matters**: Always place `RequiredPlugin` or `importPlugin` calls at the beginning of your scene definition.
    